19th Jan

Need to find the price movement when there is market overlap, between Asian, European and American markets, to explotit that information.


In [2]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
from untrade.client import Client

In [18]:
data = pd.read_csv(
    '/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/Quant_Crypto_strat/Final_gitpush/architect/processed_data.csv',
    # parse_dates=['datetime'],  # Parse the 'datetime' column as datetime
    # index_col='datetime'       # Use 'datetime' as the index
)

data = pd.read_csv(
    '/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/BTC/BTC_2019_2023_1d.csv',
    # parse_dates=['datetime'],  # Parse the 'datetime' column as datetime
    # index_col='datetime'
) 

In [9]:
import mplfinance as mpf
print(mpf.__version__)

0.12.10b0


In [ ]:
import pandas as pd
import talib
import mplfinance as mpf
import matplotlib.pyplot as plt

# Load your data (replace 'your_data.csv' with your actual file)
# data = pd.read_csv("your_data.csv")
data['datetime'] = pd.to_datetime(data['datetime'])  # Ensure datetime is properly formatted
data.set_index('datetime', inplace=True)  # Set datetime as the index for plotting

# List of candlestick patterns to detect
candlestick_patterns = {
    "Doji": talib.CDLDOJI,
    "Hammer": talib.CDLHAMMER,
    "Engulfing": talib.CDLENGULFING,
    "Marubozu": talib.CDLMARUBOZU,
    "Shooting Star": talib.CDLSHOOTINGSTAR,
}

# Detect patterns and store results
for pattern_name, pattern_function in candlestick_patterns.items():
    data[pattern_name] = pattern_function(data['open'], data['high'], data['low'], data['close'])


def plot_pattern(data, pattern_name):
    import matplotlib.pyplot as plt
    
    # Filter data where the pattern is detected
    pattern_data = data[data['signals'] == pattern_name]
    
    for row in pattern_data.itertuples():
        start_date = row.Index
        end_date = start_date + pd.Timedelta(days=10)  # Adjust for the window you want to display
        subset_data = data.loc[start_date:end_date]
        
        # Plot the candlestick chart
        mpf.plot(
            subset_data,
            type='candle',
            style='charles',
            title=f"{pattern_name} Pattern - {row.Index.date()}",
            savefig=f"{pattern_name}_{row.Index.date()}.png",
            show_nontrading=True,
        )
        plt.scatter(
            [row.Index],
            [row.close],
            color='red',
            label=f'{pattern_name}',
        )
        plt.legend()
        plt.show()

# Plot detected patterns
for pattern_name in candlestick_patterns.keys():
    plot_pattern(data, pattern_name)

# Save the full dataset with signals
data.to_csv("candlestick_analysis_with_signals.csv")

#convert this to excel file
data.to_excel("candlestick_analysis_with_signals.xlsx")

In [19]:
import pandas as pd
import talib
import mplfinance as mpf
import matplotlib.pyplot as plt

# Load your data (replace 'your_data.csv' with your actual file)
# data = pd.read_csv("your_data.csv")
data['datetime'] = pd.to_datetime(data['datetime'])  # Ensure datetime is properly formatted
data.set_index('datetime', inplace=True)  # Set datetime as the index for plotting

# List of candlestick patterns to detect
candlestick_patterns = {
    "Doji": talib.CDLDOJI,
    "Hammer": talib.CDLHAMMER,
    "Engulfing": talib.CDLENGULFING,
    "Marubozu": talib.CDLMARUBOZU,
    "Shooting Star": talib.CDLSHOOTINGSTAR,
}

# Detect patterns and store results
for pattern_name, pattern_function in candlestick_patterns.items():
    data[pattern_name] = pattern_function(data['open'], data['high'], data['low'], data['close'])

def plot_pattern(data, pattern_name):
    import matplotlib.pyplot as plt
    
    # Filter data where the pattern is detected
    pattern_data = data[data[pattern_name] != 0]  # Keep rows where pattern is detected

    for row in pattern_data.itertuples():
        start_date = row.Index
        end_date = start_date + pd.Timedelta(days=10)  # Adjust for the window you want to display
        subset_data = data.loc[start_date:end_date]

        # Plot the candlestick chart
        mpf.plot(
            subset_data,
            type='candle',
            style='charles',
            title=f"{pattern_name} Pattern - {row.Index.date()}",
            savefig=f"{pattern_name}_{row.Index.date()}.png",
            show_nontrading=True,
        )
        plt.scatter(
            [row.Index],
            [row.close],
            color='red',
            label=f'{pattern_name}',
        )
        plt.legend()
        plt.show()

# Plot detected patterns
# for pattern_name in candlestick_patterns.keys():
    # plot_pattern(data, pattern_name)

# Filter out rows where **no pattern is detected**
pattern_columns = list(candlestick_patterns.keys())
filtered_data = data[(data[pattern_columns] != 0).any(axis=1)]

# Save the filtered dataset with detected patterns
filtered_data.to_csv("fcandlestick_analysis_with_signals.csv")
filtered_data.to_excel("fcandlestick_analysis_with_signals.xlsx")

print("Filtered data saved to CSV and Excel.")

Filtered data saved to CSV and Excel.


In [13]:
#data of 31st december 2023

data_last = data.loc['2023-12-31']
print(data_last)

                     Unnamed: 0     open     high      low    close     volume
datetime                                                                      
2023-12-31 00:00:00        9446  42174.3  42478.9  42001.0  42209.4  24649.414
2023-12-31 04:00:00        9447  42209.4  42729.2  42189.9  42567.0  27687.399
2023-12-31 08:00:00        9448  42567.0  42951.0  42400.2  42561.8  37644.346
2023-12-31 12:00:00        9449  42561.8  42694.0  42381.0  42489.6  29877.289
2023-12-31 16:00:00        9450  42489.6  42766.7  42465.4  42659.9  21496.084
2023-12-31 20:00:00        9451  42659.9  42724.5  42083.1  42314.0  32153.757


In [14]:
# 12:30 PM IST to 2:30 PM IST

# 7:30 PM IST to 9:30 PM IST
#in the frame of data_day, create a new column, which consists the difference between the open at 12:30 PM everyday and the close of 2:30PM of the same dat;
#in the frame of data_day, create a one more new column, which consists the difference between the open at 7:30 PM everyday and the close of 9:30PM of the same data
#this data is available in the frame data


In [15]:
data_1230_1430 = data.between_time('12:30', '14:30')
data_1930_2130 = data.between_time('19:30', '21:30')

diff_1230_1430 = data_1230_1430.groupby(data_1230_1430.index.date).apply(
    lambda x: x.iloc[-1]['close'] - x.iloc[0]['open']
)
diff_1930_2130 = data_1930_2130.groupby(data_1930_2130.index.date).apply(
    lambda x: x.iloc[-1]['close'] - x.iloc[0]['open']
)

diff_1230_1430 = pd.Series(diff_1230_1430.values, index=pd.to_datetime(diff_1230_1430.index))
diff_1930_2130 = pd.Series(diff_1930_2130.values, index=pd.to_datetime(diff_1930_2130.index))

data_day['diff_1230_1430'] = pd.Series(diff_1230_1430)
data_day['diff_1930_2130'] = pd.Series(diff_1930_2130)

data_day['diff_open_close'] = data_day['close'] - data_day['open']

data_day_new = data_day[['diff_1230_1430', 'diff_1930_2130', 'diff_open_close']].copy()

print(data_day_new)


ValueError: Data must be 1-dimensional, got ndarray of shape (0, 6) instead

There is some error with the day 1 data,because of the nan value

this dataframe holds the price movement between the overlap of the markets.

dropping the first day data because of the nan value

In [9]:
#gfetting the last day data;;
last_day_data = data_day_new.iloc[-1]
print(last_day_data)

NameError: name 'data_day_new' is not defined

In [7]:
import plotly.graph_objects as go


In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_day_new.index,
    y=data_day_new['diff_1230_1430'],
    mode='lines',
    name='diff_1230_1430'
))

fig.add_trace(go.Scatter(
    x=data_day_new.index,
    y=data_day_new['diff_1930_2130'],
    mode='lines',
    name='diff_1930_2130'
))

fig.add_trace(go.Scatter(
    x=data_day_new.index,
    y=data_day_new['diff_open_close'],
    mode='lines',
    name='diff_open_close'
))

fig.update_layout(
    title="BTC Price Differences (12:30-14:30, 19:30-21:30, Open-Close)",
    xaxis_title="Date",
    yaxis_title="Price Difference",
    template="plotly_dark"
)

fig.show()

NameError: name 'data_day_new' is not defined